# Audio Adapter Tests
This notebook contains unit

In [ ]:
import unittest
import os
import tempfile
from unittest.mock import patch, MagicMock
import numpy as np
import soundfile as sf

# Add the parent directory to the path so we can import the adapters
import sys
sys.path.append('..')
from adapters.audio

In [ ]:
class AudioAdapterTest(unittest.TestCase):
    def setUp(self):
        # Create a temporary directory for test files
        self.test_dir = tempfile.mkdtemp()

        # Create a simple test audio signal
        self.sample_rate = 44100  # 44.1 kHz
        self.duration = 0.5  # 0.5 seconds
        t = np.linspace(0, self.duration, int(self.sample_rate * self.duration), endpoint=False)
        self.test_audio = 0.5 * np.sin(2 * np.pi * 440 * t)  # 440 Hz sine wave at half amplitude

        # Create a test audio file
        self.test_file = os.path.join(self.test_dir, "test_audio.wav")
        sf.write(self.test_file, self.test_audio, self.sample_rate)

    def tearDown(self):
        # Clean up test files
        if os.path.exists(self.test_file):
            os.remove(self.test_file)
        if os.path.exists(self.test_dir):
            os.rmdir(self.test_dir)

    def test_initialization(self):
        """Test basic initialization of AudioAdapter"""
        adapter = AudioAdapter({})
        self.assertIsNotNone(adapter)
        self.assertEqual(adapter.config, {})

    def test_load_audio(self):
        """Test loading audio from file"""
        adapter = AudioAdapter({})
        adapter.load(self.test_file)

        # Check if the audio data was loaded correctly
        self.assertIsNotNone(adapter.audio_data)
        self.assertEqual(adapter.sample_rate, self.sample_rate)
        np.testing.assert_allclose(adapter.audio_data, self.test_audio, rtol=1e-5)

    def test_generate_audio(self):
        """Test generating audio with the adapter"""
        adapter = AudioAdapter({})

        # Generate a sine wave
        adapter.generate_tone(frequency=440, duration=0.5, amplitude=0.5)

        # Check if audio was generated correctly
        self.assertIsNotNone(adapter.audio_data)
        self.assertEqual(len(adapter.audio_data), int(adapter.sample_rate * 0.5))

    def test_apply_effects(self):
        """Test applying effects to audio data"""
        adapter = AudioAdapter({})
        adapter.load(self.test_file)

        original_audio = adapter.audio_data.copy()

        # Apply gain effect
        adapter.apply_gain(2.0)  # Double the amplitude

        # Check if gain was applied correctly
        self.assertIsNotNone(adapter.audio_data)
        np.testing.assert_allclose(adapter.audio_data, original_audio * 2.0, rtol=1e-5)

    def test_save_audio(self):
        """Test saving audio to file"""
        adapter = AudioAdapter({})
        adapter.generate_tone(frequency=440, duration=0.5, amplitude=0.5)

        # Save to a new file
        output_file = os.path.join(self.test_dir, "output_audio.wav")
        adapter.save(output_file)

        # Check if file was created
        self.assertTrue(os.path.exists(output_file))

        # Load the saved file and check its contents
        data, rate = sf.read(output_file)
        self.assertEqual(rate, adapter.sample_rate)
        np.testing.assert_allclose(data, adapter.audio_data, rtol=1e-5)

        # Clean up
        os.remove(output_file)

    @patch('soundfile.write')
    def test_execute(self, mock_write):
        """Test execute method"""
        adapter = AudioAdapter({})
        adapter.generate_tone(frequency=440, duration=0.5, amplitude=0.5)
        adapter.output_file("test_output.wav")

        # Execute the adapter
        result = adapter.execute()

        # Check if file was written
        mock_write.assert_called_once()
        self.assertEqual(result, "Audio processed and saved to test

In [ ]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)